In [2]:
import pandas as pd
import re

In [16]:
def preprocess(data):
    pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

    messages = re.split(pattern, data)[1:]
    dates = re.findall(pattern, data)

    df = pd.DataFrame({'user_message': messages, 'message_date': dates})
    # convert message_date type
    df['message_date'] = pd.to_datetime(df['message_date'], format='%m/%d/%y, %H:%M - ')

    df.rename(columns={'message_date': 'date'}, inplace=True)

    users = []
    messages = []
    for message in df['user_message']:
        entry = re.split('([\w\W]+?):\s', message)
        if entry[1:]:  # user name
            users.append(entry[1])
            messages.append(" ".join(entry[2:]))
        else:
            users.append('group_notification')
            messages.append(entry[0])

    df['user'] = users
    df['message'] = messages
    df.drop(columns=['user_message'], inplace=True)

    df['only_date'] = df['date'].dt.date
    df['year'] = df['date'].dt.year
    df['month_num'] = df['date'].dt.month
    df['month'] = df['date'].dt.month_name()
    df['day'] = df['date'].dt.day
    df['day_name'] = df['date'].dt.day_name()
    df['hour'] = df['date'].dt.hour
    df['minute'] = df['date'].dt.minute

    period = []
    for hour in df[['day_name', 'hour']]['hour']:
        if hour == 23:
            period.append(str(hour) + "-" + str('00'))
        elif hour == 0:
            period.append(str('00') + "-" + str(hour + 1))
        else:
            period.append(str(hour) + "-" + str(hour + 1))

    df['period'] = period

    return df

In [17]:
with open("K:\code\chat_analyser\WhatsApp Chat with Home.txt", "r", encoding="utf-8") as f:
    data = f.read()

df = preprocess(data)


In [18]:
df.head()

,date,user,message,only_date,year,month_num,month,day,day_name,hour,minute,period
0,2025-06-29 15:08:00,group_notification,Messages and calls are end-to-end encrypted. O...,2025-06-29,2025,6,June,29,Sunday,15,8,15-16
1,2025-06-29 15:08:00,group_notification,"vasudev cse created group ""Home""\n",2025-06-29,2025,6,June,29,Sunday,15,8,15-16
2,2025-06-29 15:08:00,group_notification,vasudev cse added you\n,2025-06-29,2025,6,June,29,Sunday,15,8,15-16
3,2025-06-29 15:09:00,vasudev cse,POLL Bhai wifi rakhna ho toh abhi recharge k...,2025-06-29,2025,6,June,29,Sunday,15,9,15-16
4,2025-06-29 15:09:00,group_notification,vasudev cse pinned a message\n,2025-06-29,2025,6,June,29,Sunday,15,9,15-16
